In [1]:
import pandas as pd
import os

First step is to create the connection to Databricks

In [5]:
# Connect to Databricks
from databricks import sql
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pandas as pd

# Load the environment variables
load_dotenv()

# Set the connection
connection =  sql.connect(server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME"),
                 http_path       = os.getenv("DATABRICKS_HTTP_PATH"),
                 access_token    = os.getenv("PYTHON_ACCESS_TOKEN"))

cursor = connection.cursor()


# Start spark session
spark = SparkSession.builder \
    .appName("mkt_analysis_2024") \
    .getOrCreate()

def get_df_from_databricks(query):
    # Execute the query
    cursor.execute(query)
    # Get the result
    result = cursor.fetchall()
    # Get the column names
    columns = [desc[0] for desc in cursor.description]
    # Create a pandas DataFrame
    df = pd.DataFrame(result, columns=columns)
    return df


24/05/16 07:59:08 WARN Utils: Your hostname, Bernardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.117 instead (on interface en0)
24/05/16 07:59:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 07:59:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


A bit about how the data is structured at Metaloop. We essentially have these sources of trasactional material data:

 - Cart items
    - These are the materials that compose a transaction (stored in the orders object). Each order is associated with two accounts, a seller and a buyer. Therefore, the lowest granularity data here will be cart item level data with data from the orders joined
 - Opportunity materials
    - These paly a similar role as cart items, but on the opportunity level (prior to an order being put forward)
 - Conditions (Contract materials)

 

In [7]:
# Read the query from the query file    
queries_path = os.path.join(os.getcwd(), "queries/")

# Read the cart items query
query = open(queries_path + "cart_items.sql").read()

# Get cart_items
cart_items = get_df_from_databricks(query)

/Users/bernardocarvalho/Desktop/imperial_repos/work_based_project/venv/lib/python3.11/site-packages/IPython/core/formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/Users/bernardocarvalho/Desktop/imperial_repos/work_based_project/venv/lib/python3.11/site-packages/IPython/core/formatters.py:347: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,pk_cart_item,fk_order,ds_material_alias,ds_material,fk_material_alias,fk_material,ds_purchase_price_formula,ds_sale_price_formula,vl_quantity_purchase,vl_quantity_sell,vl_unit_price_purchase,vl_unit_price_sell
0,a0V09000000p8gCEAQ,a0b09000000gKg6AAE,Cu mit Fe (87%),"Kupfer, gemischt, Raff, alt",a0g09000001F4tuAAC,a0Z09000005ERBFEA4,None,None,374.00,374.00,4.380000,4.380000
1,a0V09000000p8vyEAA,a0b09000000gKgBAAU,"Kupfer, blank I, Millberry","Kupfer, blank I, Millberry",a0g09000001F4tqAAC,a0Z09000000oVReEAM,None,None,220.00,220.00,5.150000,5.150000
2,a0V09000000p8vzEAA,a0b09000000gKgBAAU,"Kupfer schwer, Berry min. 98% Cu","Kupfer schwer, Berry min. 98% Cu",a0g09000001F4tmAAC,a0Z09000005ERBKEA4,None,None,25.00,25.00,4.430000,4.430000
3,a0V09000000p8w0EAA,a0b09000000gKgBAAU,Kupferkabel mit Stecker,Kupferkabel mit Stecker,a0g09000001F4uMAAS,a0Z09000005ESrsEAG,None,None,5.00,5.00,0.670000,0.670000
4,a0V09000000p8w1EAA,a0b09000000gKgBAAU,"Kupferkabel, min. 38% Cu","Kupferkabel, min. 38% Cu",a0g09000001F4uOAAS,a0Z09000000oVSIEA2,None,None,50.00,50.00,1.410000,1.410000
